In [1]:
import psycopg2

In [2]:
import logging
import sys

logging.basicConfig(stream = sys.stdout, level=logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [3]:
# Connect to database

def create_connection():
    connection = psycopg2.connect(
        host="introduction-01-intro-ap-southeast-1-dev-introduction-db.cpfm8ml2cxp2.ap-southeast-1.rds.amazonaws.com",
        database="postgres",
        user="postgres",
        password="postgres123"
    )
    connection.set_session(autocommit=True)
    return connection

connection = create_connection()
logger.info(f"connection log: {connection}")

INFO:root:connection log: <connection object at 0x72614d928b80; dsn: 'user=postgres password=xxx dbname=postgres host=introduction-01-intro-ap-southeast-1-dev-introduction-db.cpfm8ml2cxp2.ap-southeast-1.rds.amazonaws.com', closed: 0>


In [40]:
event = {
    "urls" : [
        "https://phimmoiyyy.net/the-loai/phim-chieu-rap",
        "https://phimmoiiii.net/the-loai/phim-co-trang",
        "https://phimmoiiii.net/the-loai/phim-hoat-hinh"

    ],
    "table_name": "ai4e_movie_giakhanh",
    "data_field": [
        {
            "field": "id",
            "type": "integer"
        },
        {
            "field": "movie_link",
            "type": "varchar"
        },
        {
            "field": "poster_image_link",
            "type": "varchar"
        },
        {
            "field":"movie_name",
            "type" : "varchar"
        },
        {
            "field": "movie_status",
            "type": "varchar"
        },
        {
            "field": "en_name",
            "type": "varchar"
        },
        {
            "field": "vn_name",
            "type": "varchar"
        },
        {
            "field": "category",
            "type": "varchar"
        }
    ]
}

In [41]:
# Create a table in the database we connected 

def create_table(connection, table_name, fields):
    logger.info(f"Start create table {table_name}")
    list_field_name_field_type = [f'{field["field"]} {field["type"]}' for field in fields]
    result_field_name_field_type = ','.join(list_field_name_field_type)

    create_table_str = f"""
        create table if not exists {table_name} (
            {result_field_name_field_type}
        )
    """

    cursor = connection.cursor()
    cursor.execute(create_table_str)


table_name = "ai4e_movie_giakhanh"
fields = event["data_field"]

create_table(connection, table_name, fields)

INFO:root:Start create table ai4e_movie_giakhanh


In [42]:
import requests
import bs4

In [43]:
total_movies = []

for url in event["urls"]:
    info = requests.get(url, headers={})

    
    soup = bs4.BeautifulSoup(info.content, 'lxml')

    movies = soup.find_all("article", {"class": "item movies"})
    category = soup.find("div", {"class" : "content right normal"}).find("h1").text
    print(len(movies))

    for movie in movies:
        # Take movid_id
        movies_id = movie["id"].split("-")[1]

        # Take movie_link
        movie_footer = movie.find('div', {"class": "data"})
        movie_link = movie_footer.find("a")["href"]

        # Take poster_image_link
        poster_image_link = movie.find("img")["src"]

        # Take en_name
        span_tag = movie.find("span").find("span")
        name_with_year = span_tag.text
        name_parts = name_with_year.split()[:-1]
        en_name = " ".join(name_parts)

        #Take movie_status
        movie_status = movie.find("div", {"class":"trangthai"}).text

        #Take vn_name
        vn_name = movie.find("div",{"class":"data"}).find("a").text

        #Take movie_name 
        movie_name = movie.find("img")["alt"]

        #Take category
        category = category

        movie_dict = {
            "id": movies_id,
            "movie_link": movie_link,
            "poster_image_link": poster_image_link,
            "movie_name": movie_name,
            "movie_status": movie_status,
            "en_name": en_name,
            "vn_name": vn_name,
            "category": category
        }
        total_movies.append(movie_dict)

print(total_movies)

29
21
12
[{'id': '16155', 'movie_link': 'https://phimmoiiii.net/phim-le/bua-hinh-nhan', 'poster_image_link': 'https://phimmoiiii.net/wp-content/uploads/2023/04/Bua-Hinh-Nhan.jpg', 'movie_name': 'Bùa Hình Nhân', 'movie_status': 'HD Vietsub', 'en_name': 'Hoon Payon', 'vn_name': 'Bùa Hình Nhân', 'category': 'Phim Chiếu Rạp'}, {'id': '34569', 'movie_link': 'https://phimmoiiii.net/phim-le/hon-ma-khong-dau', 'poster_image_link': 'https://phimmoiiii.net/wp-content/uploads/2022/07/hon-ma-khong-dau.jpg', 'movie_name': 'Hồn Ma Không Đầu', 'movie_status': 'HD Vietsub', 'en_name': 'Ivanna', 'vn_name': 'Hồn Ma Không Đầu', 'category': 'Phim Chiếu Rạp'}, {'id': '3208', 'movie_link': 'https://phimmoiiii.net/phim-le/ma-lai-rut-ruot', 'poster_image_link': 'https://phimmoiiii.net/wp-content/uploads/2023/04/Phim-Ma-Lai-Rut-Ruot.jpg', 'movie_name': 'Ma Lai Rút Ruột', 'movie_status': 'HD Vietsub', 'en_name': 'Inhuman Kiss 2', 'vn_name': 'Ma Lai Rút Ruột', 'category': 'Phim Chiếu Rạp'}, {'id': '31855', 'movi

In [48]:
# Process the single quote in data

for i in total_movies:
    for k, v in i.items():
        i[k] = v.replace("'", "''")

In [50]:
# Insert to database

def insert_database(connection, table_name, data_movies):
    logger.info(f"Start insert data into table {table_name}")
    list_field_name = [f'{field["field"]}' for field in event["data_field"]]
    result_field_name = ','.join(list_field_name)

    data_insert = [f"({item['id']}, '{item['movie_link']}', '{item['poster_image_link']}', '{item['movie_name']}', '{item['movie_status']}', '{item['en_name']}', '{item['vn_name']}', '{item['category']}')" for item in total_movies]
    
    result_insert = ",".join(data_insert)
    
    sql_insert = f"""
        INSERT INTO {table_name} ({result_field_name})
        VALUES {result_insert};
    """
    print(sql_insert)
    cursor = connection.cursor()
    cursor.execute(sql_insert)

insert_database(connection, table_name, total_movies)

INFO:root:Start insert data into table ai4e_movie_giakhanh

        INSERT INTO ai4e_movie_giakhanh (id,movie_link,poster_image_link,movie_name,movie_status,en_name,vn_name,category)
        VALUES (16155, 'https://phimmoiiii.net/phim-le/bua-hinh-nhan', 'https://phimmoiiii.net/wp-content/uploads/2023/04/Bua-Hinh-Nhan.jpg', 'Bùa Hình Nhân', 'HD Vietsub', 'Hoon Payon', 'Bùa Hình Nhân', 'Phim Chiếu Rạp'),(34569, 'https://phimmoiiii.net/phim-le/hon-ma-khong-dau', 'https://phimmoiiii.net/wp-content/uploads/2022/07/hon-ma-khong-dau.jpg', 'Hồn Ma Không Đầu', 'HD Vietsub', 'Ivanna', 'Hồn Ma Không Đầu', 'Phim Chiếu Rạp'),(3208, 'https://phimmoiiii.net/phim-le/ma-lai-rut-ruot', 'https://phimmoiiii.net/wp-content/uploads/2023/04/Phim-Ma-Lai-Rut-Ruot.jpg', 'Ma Lai Rút Ruột', 'HD Vietsub', 'Inhuman Kiss 2', 'Ma Lai Rút Ruột', 'Phim Chiếu Rạp'),(31855, 'https://phimmoiiii.net/phim-le/ma-so-truc-quy', 'https://phimmoiiii.net/wp-content/uploads/2022/11/ma-so-truc-quy.jpg', 'Ma Sơ Trục Quỷ', 'Coming So